# Can I Beat the Stock Market? A Simulation and Analysis

I have always been intrigued by the idea of being an active stock investor. The allure of developing a strategy to beat the market with some fancy algorithm while I travel the world with my family seems pretty great! To this end, I recently started the [Machine Learning for Trading](https://www.udacity.com/course/machine-learning-for-trading--ud501) course at Udacity. One of the concepts covered early on in this class are Bollinger Bands (BB). BBs use a 20-day rolling mean of the stock price to compute the an upper and lower price band indicating the price that is two standard deviations above or below the rolling mean. When the daily price exceeds these bands it indicates that the price is relatively high or low and is likely to regress towards the 20-day mean. I could see the riches in my mind's eye - it seems like such an easy way to execute the age old mantra of "buy low, sell high". When price dips below the lower band, buy and when it rises past the upper band, sell. Easy, right?

In [1]:
import quandlkey
import pandas as pd

In [2]:
# pick a list of stocks to use in the simulation
stocks = quandlkey.stocks()[0].sample(49).str[5:]
stocks

2024     OHRP
155       EQR
1032     CNOB
1537      IRG
615       AFG
1599      IRF
410       SLM
2640     TRST
2243     ROLL
542       ACM
1019      CTG
2632      TRN
78        CAG
2354      SWM
339      ORLY
808       BCC
2460     STMP
633      AMPE
3133      ORM
2332     SAFT
505       AAN
125       DHR
1069       CR
437       TSN
606      UHAL
2252     RCPT
360       PFE
489       YUM
2189     PSMT
977      CHCO
1389       GB
2918      ESS
1952     NEON
1511     HSNI
1896     MOVE
300       MON
23        AFL
1562    IMKTA
2025      OIS
1047      CPA
1650      KAI
927     CNBKA
1302      FGL
2882      AEC
135       DOW
533        AE
973      CTRN
2971     FULT
2208     PTCT
Name: 0, dtype: object

1514    JBHT
1272     XLS
117     CSCO
484       WU
600      AMC
2624    TRMR
343     NWSA
356     PDCO
1088    CMLS
1396     GEF
1210    ECPG
572     ALGN
1301    FCSC
2846    XOOM
2745    VICR
1131    DHIL
883      CRR
1558    INFA
1821    MEAS
2762    VOCS
1501     HMN
2361     SMG
1012     CBU
101      COL
1079     CSS

In [3]:
# pick a date range for which the stocks are available, perhaps use relatively recent tiem frame, 3-5 years
start_date = (2015, 1, 1)

In [4]:
# strategy
baseresults = []
myresults = []
params = []
company = []
starts = []
ends = []
windows = [2,5,10,20,25,30,40,50]
sell_pers = [0.1,0.25,0.50,0.75,0.90]
buy_amts = [50,100,250,500,750]
widths = [1,2,3]
initial_balance = 1000
sim_num = 0
for stock in stocks:
    stock_data = quandlkey.quandl_stocks(stock, start_date=start_date)
    try:
        for window in windows:
            for sell_per in sell_pers:
                for buy_amt in buy_amts:
                    for width in widths:
                        while sim_num < 10:
                            start = stock_data['WIKI/{} - Adj. Close'.format(stock)].sample(1).index[0]
                            end = stock_data['WIKI/{} - Adj. Close'.format(stock)][start:].sample(1).index[0]
                            sim = quandlkey.BollingerBands(stock_data['WIKI/{} - Adj. Close'.format(stock)].loc[start:end], 
                                           window=window, sell_per=sell_per, buy_amt=buy_amt, width=width,
                                            balance=initial_balance)
                            sim.trade()
                            base = sim.buy_and_hold()
                            baseresults.append(base[-1])
                            thisreturn = sim.results['Shares']*sim.stock+sim.results['Balance']
                            myresults.append(thisreturn[-1])
                            params.append((window, sell_per, buy_amt, width))
                            company.append(stock)
                            starts.append(start)
                            ends.append(end)
        
                            sim_num += 1
                        sim_num = 0
    except:
        pass

In [5]:
# convert params list of tuples into a DataFrmae
params = pd.DataFrame(params, columns=['Window','SellPer','BuyAmt','Width'])

In [6]:
paramSim = pd.DataFrame({'Base':baseresults,'Maxi':myresults, 'Company':company, 'Start':starts, 'End':ends})
paramSim = pd.concat([paramSim, params], axis=1)
# convert dollar returns to percentages
paramSim[['Base','Maxi']] = (paramSim[['Base','Maxi']]/initial_balance)-1 # converts dollar returns to % returns

In [7]:
paramSim.to_csv('Bollinger Simulation Results.csv')

### Analysis
1. What was the average return for the two strategies?
    - Why: compares random stock investing to a strategy
2. Did any combination of parameters reliably beat the market?
    - Why: AAPL/GE analysis indicate that only a fraction of time my stategy beats the market. Do one of the other combinations work better? (whole point of doing this grid search)
3. Where there any specific companies that my strategy worked on?
    - Why: Would indicate that the strategy only works for certain types of companies.
    - Follow up to this result would be to test on other similar companies. Another project stemming from this would be to build an algorithm that could predict when my strategy would work.
    - Would likely need to run the simulation again and keep track of the start/end dates in order to understand the market trend that leads to success. Would actually need to get x days of data leading up to time region I perform the simulation on. So the training data would be something like 1 month of data on the data and then simulate on the following year of data. Allows to answer question, will this stock return a certain level over the next year?

In [8]:
paramSim = pd.read_csv('Bollinger Simulation Results.csv', index_col=0)

1. What was the average return for the two strategies?
    - Why: compares random stock investing to a strategy
    - [Likely Need to do Wilcoxon Matched Pairs Test](http://www.biochemia-medica.com/content/comparing-groups-statistical-differences-how-choose-right-statistical-test)

In [9]:
# quick histogram, requires some serious tuning for good viz, the skew of the returns using my strategy is crazy
# tells us what type of distribution we are dealing with which informs which measure of central tendency to use
# for comparing the strategies
%matplotlib
fig, ax = plt.subplots(figsize=(16,9))
paramSim['Base'].plot(kind='hist', ax=ax, legend=True, fontsize=20)
paramSim['Maxi'].plot(kind='hist', bins=1000, ax=ax, legend=True, fontsize=20, alpha=0.5);
#ax.set_ylim(0,8000)
ax.set_xlim(-3,20)

Using matplotlib backend: Qt5Agg


(-3, 20)

The distribution of the returns are interestingly shaped. The Base values are tightly centered around 0 with max and min values of 1 and -1, respectively. The Maxi data however, is highly skewed. The minimum value is -1 while the maximum value is over 1000. The median and mean investment returns are plotted below.

In [10]:
def iqr(x):
    """x 1D array/Series"""
    # calculate interquartile range
    q25, q75 = np.percentile(x, 25), np.percentile(x, 75)
    iqr = q75 - q25
    return iqr

In [11]:
%matplotlib
# calculate interquartile range
iqr_b = iqr(paramSim['Base'])
iqr_m = iqr(paramSim['Maxi'])
f, a = plt.subplots(figsize=(16,9))
paramSim[['Base','Maxi']].median().plot(kind='bar', yerr=[iqr_b,iqr_m], ax=a,
                                      fontsize=20, rot=0)
a.set_title('Median Investment Return', fontsize=20)
a.set_ylabel('% Return/100', fontsize=20)
a.set_xlabel('Strategy', fontsize=20)
a.text(0.1,.05,'{:.3f}%'.format(paramSim['Base'].median()*100), fontdict={'fontsize':20})
a.text(1.05,.05,'{:.3f}%'.format(paramSim['Maxi'].median()*100), fontdict={'fontsize':20});

Using matplotlib backend: Qt5Agg


In [12]:
%matplotlib
f, a = plt.subplots(figsize=(16,9))
paramSim[['Base','Maxi']].mean().plot(kind='bar', yerr=paramSim[['Base','Maxi']].sem(), ax=a,
                                      fontsize=20, rot=0)
a.set_title('Mean Investment Return', fontsize=20)
a.set_ylabel('% Return/100', fontsize=20)
a.set_xlabel('Strategy', fontsize=20)
a.text(0-.045,0.3,'{:.2f}%'.format(paramSim['Base'].mean()*100), fontdict={'fontsize':20})
a.text(.95,3.0,'{:.2f}%'.format(paramSim['Maxi'].mean()*100), fontdict={'fontsize':20});

Using matplotlib backend: Qt5Agg


Since the distributions do not look normal, we'll compare the returns from each method using the Wilcoxon signed-rank test. This test is a non-parametric version of the paired t-test. We are using the paired t-test because we care about the difference between the two strategie for each observation.

In [13]:
from scipy.stats import wilcoxon
from scipy import stats
from scipy.stats import distributions

In [14]:
def wilcoxon_(x, y=None, zero_method="wilcox", correction=False):
    """
    Calculate the Wilcoxon signed-rank test.
    The Wilcoxon signed-rank test tests the null hypothesis that two
    related paired samples come from the same distribution. In particular,
    it tests whether the distribution of the differences x - y is symmetric
    about zero. It is a non-parametric version of the paired T-test.
    Parameters
    ----------
    x : array_like
        The first set of measurements.
    y : array_like, optional
        The second set of measurements.  If `y` is not given, then the `x`
        array is considered to be the differences between the two sets of
        measurements.
    zero_method : string, {"pratt", "wilcox", "zsplit"}, optional
        "pratt":
            Pratt treatment: includes zero-differences in the ranking process
            (more conservative)
        "wilcox":
            Wilcox treatment: discards all zero-differences
        "zsplit":
            Zero rank split: just like Pratt, but spliting the zero rank
            between positive and negative ones
    correction : bool, optional
        If True, apply continuity correction by adjusting the Wilcoxon rank
        statistic by 0.5 towards the mean value when computing the
        z-statistic.  Default is False.
    Returns
    -------
    T : float
        The sum of the ranks of the differences above or below zero, whichever
        is smaller.
    p-value : float
        The two-sided p-value for the test.
    Notes
    -----
    Because the normal approximation is used for the calculations, the
    samples used should be large.  A typical rule is to require that
    n > 20.
    References
    ----------
    .. [1] http://en.wikipedia.org/wiki/Wilcoxon_signed-rank_test
    """

    if not zero_method in ["wilcox", "pratt", "zsplit"]:
        raise ValueError("Zero method should be either 'wilcox' \
                          or 'pratt' or 'zsplit'")

    if y is None:
        d = x
    else:
        x, y = map(np.asarray, (x, y))
        if len(x) != len(y):
            raise ValueError('Unequal N in wilcoxon.  Aborting.')
        d = x-y

    if zero_method == "wilcox":
        d = np.compress(np.not_equal(d, 0), d, axis=-1)  # Keep all non-zero differences

    count = len(d)
    if (count < 10):
        warnings.warn("Warning: sample size too small for normal approximation.")
    r = stats.rankdata(abs(d))
    r_plus = np.sum((d > 0) * r, axis=0)
    r_minus = np.sum((d < 0) * r, axis=0)

    if zero_method == "zsplit":
        r_zero = np.sum((d == 0) * r, axis=0)
        r_plus += r_zero / 2.
        r_minus += r_zero / 2.

    T = min(r_plus, r_minus)
    mn = count*(count + 1.) * 0.25
    se = count*(count + 1.) * (2. * count + 1.)

    if zero_method == "pratt":
        r = r[d != 0]

    replist, repnum = stats.find_repeats(r)
    if repnum.size != 0:
        # Correction for repeated elements.
        se -= 0.5 * (repnum * (repnum * repnum - 1)).sum()

    se = np.sqrt(se / 24)
    correction = 0.5 * int(bool(correction)) * np.sign(T - mn)
    z = (T - mn - correction) / se
    prob = 2. * distributions.norm.sf(abs(z))
    return T, prob, z
T, p, z = wilcoxon_(paramSim['Base'],paramSim['Maxi'])
print(T, p, z)

10410007282.5 0.0 -231.195824242


Above, I modified the scipy.stats.wilcoxon function to return the Z value. This allows for the calculation of an effect size. The p-value indicates a significant effect, however, the number of samples we are dealing with is so large that the p-value is likely meaningless. We can use the Z vale to calculate the [effect size](https://stats.stackexchange.com/questions/133077/effect-size-to-wilcoxon-signed-rank-test).

In [15]:
print("Cohen's d: {:.2f}".format(z/np.sqrt(len(paramSim))))

Cohen's d: -0.43


A Cohen's d with a magnitude of 0.1 is considered a small effect size. The negative value means that there was an increase in the values in the Maxi strategy compared to the baseline strategy. These results indicate that for a given observation, the Maxi strategy gives a better return then randomly investing in a particular stock. This is driven home by considering the mode of the difference in the returns by the two strategies:

In [16]:
# use the describe function get some basic info
(paramSim['Maxi']-paramSim['Base']).std()

0.49476697531285585

The most common difference is a 3% greater return with the Maxi strategy. Now, we need to keep in mind that the results are **highly** variable. The standard deviation of the difference in returns is roughly 2800%. It would be nice to know if there was an implemntation of the strategy that provided greater returns without nearly so much risk:

- Did any combination of parameters reliably beat the market?
    - Why: AAPL/GE analysis indicate that only a fraction of time my stategy beats the market. Do one of the other combinations work better? (whole point of doing this grid search)

In [17]:
%matplotlib
paramSim.groupby(['Width','SellPer','BuyAmt','Window'])[['Base','Maxi']].mean().plot(kind='bar', yerr=paramSim.groupby(['Width','SellPer','BuyAmt','Window'])[['Base','Maxi']].sem())

Using matplotlib backend: Qt5Agg


In [18]:
paramSim.groupby(['Width','SellPer','BuyAmt','Window'])[['Base','Maxi']].describe().iloc[193,:]

Base  count    480.000000
      mean       0.036788
      std        0.329296
      min       -0.969016
      25%       -0.095080
      50%        0.002643
      75%        0.160123
      max        1.858938
Maxi  count    480.000000
      mean      -0.626225
      std        0.366549
      min       -1.009988
      25%       -0.971957
      50%       -0.732471
      75%       -0.284452
      max        0.025080
Name: (1, 0.9, 750, 5), dtype: float64

In [19]:
%matplotlib
paramSim[(paramSim['Width']==1)&(paramSim['SellPer']==0.9)&(paramSim['BuyAmt']==750)&(paramSim['Window']==10)]\
[['Base','Maxi']].hist(bins=100);

Using matplotlib backend: Qt5Agg


### Question to answer, what percent of the time do you get a certain level of return?

In [20]:
paramSim['Diff'] = paramSim['Maxi'] - paramSim['Base']

In [21]:
paramSim.groupby(['Width','SellPer','BuyAmt','Window'])['Diff'].describe().iloc[193,:]

count    480.000000
mean      -0.663012
std        0.528892
min       -2.818677
25%       -1.032265
50%       -0.636317
75%       -0.166210
max        0.446653
Name: (1, 0.9, 750, 5), dtype: float64

In [22]:
%matplotlib
paramSim[(paramSim['Width']==1)&(paramSim['SellPer']==0.9)&(paramSim['BuyAmt']==750)&(paramSim['Window']==10)]\
['Diff'].hist(bins=100);

Using matplotlib backend: Qt5Agg


In [23]:
# how often Maxi beats Base in general
len(paramSim[paramSim['Diff']>0])/len(paramSim)

0.36681597222222223

In general, I only beat the market 47.5% of the time (original), 45.5% in the second, larger, sim.

What is the percentage of time that Maxi beats the market with optimized parameters?

In [24]:
print('% time beat the market: ', len(paramSim[(paramSim['Width']==1)\
             &(paramSim['SellPer']==0.9)\
             &(paramSim['BuyAmt']==750)\
             &(paramSim['Window']==10)\
             &(paramSim['Diff']>0)])/\
len(paramSim[(paramSim['Width']==1)\
             &(paramSim['SellPer']==0.9)\
             &(paramSim['BuyAmt']==750)\
             &(paramSim['Window']==10)]))
print('Average return: ', paramSim[(paramSim['Width']==1)\
             &(paramSim['SellPer']==0.9)\
             &(paramSim['BuyAmt']==750)\
             &(paramSim['Window']==10)]['Diff'].median())

% time beat the market:  0.16458333333333333
Average return:  -0.4666699999999999


How often is my strategy profitable?

In [25]:
print('% time strategy is profitable: ', len(paramSim[(paramSim['Width']==1)\
             &(paramSim['SellPer']==0.9)\
             &(paramSim['BuyAmt']==750)\
             &(paramSim['Window']==10)\
             &(paramSim['Maxi']>0)])/\
len(paramSim[(paramSim['Width']==1)\
             &(paramSim['SellPer']==0.9)\
             &(paramSim['BuyAmt']==750)\
             &(paramSim['Window']==10)]))
print(len(paramSim[(paramSim['Width']==1)\
             &(paramSim['SellPer']==0.9)\
             &(paramSim['BuyAmt']==750)\
             &(paramSim['Window']==10)\
             &(paramSim['Maxi']>0)]))

% time strategy is profitable:  0.022916666666666665
11


How often does my strategy have a return of greater than 100%?

In [26]:
print('% time strategy has returns greater than 100%: ', len(paramSim[(paramSim['Width']==1)\
             &(paramSim['SellPer']==0.9)\
             &(paramSim['BuyAmt']==750)\
             &(paramSim['Window']==10)\
             &(paramSim['Maxi']>1)])/\
len(paramSim[(paramSim['Width']==1)\
             &(paramSim['SellPer']==0.9)\
             &(paramSim['BuyAmt']==750)\
             &(paramSim['Window']==10)]))

% time strategy has returns greater than 100%:  0.0


The answer is that my trading stategy beats the market 92% of the time and 77.6% of the time the return is profitable! In fact is more than 100% profitable 75% of the time. More so, on average (median) I beat the market by 3152%.

Next steps: What is the return broken down by company? Were the returns this high for each company? Then make some plots using the some of the companies/time frames that were tested.

In [27]:
best_strategy = paramSim[(paramSim['Width']==1)&(paramSim['SellPer']==0.9)&(paramSim['BuyAmt']==750)&(paramSim['Window']==10)]

In [28]:
best_strategy.groupby('Company')[['Maxi','Base','Diff']].mean()

,Maxi,Base,Diff
Company,,,
AAN,-0.538594,0.253695,-0.792289
ACM,-0.491588,-0.007996,-0.483592
AE,-0.487762,0.024100,-0.511862
AEC,-0.028968,0.000872,-0.029840
AFG,-0.750762,0.228604,-0.979366
AFL,-0.355718,-0.025840,-0.329878
AMPE,-0.351338,-0.069586,-0.281753
BCC,-0.307845,0.088328,-0.396173
CAG,-0.423708,-0.055573,-0.368134


It looks my strategy was equally successfule for all companies. I a couple companies SPF, JDSU, HTWR, and ARO that have much lower returns the other companies, on average. Keep in mind that those companies are still delivering 700% returns. The fact that different companies have different returns has me thinking that we could have a fun machine learning project that uses the historical stock information to predict the success of my strategy on a particular company.

For now, let's take a look at a particular company and see if any thing is there.

In [30]:
best_strategy[best_strategy['Company']=='YUM']

,Base,Company,End,Maxi,Start,Window,SellPer,BuyAmt,Width,Diff
164220,-0.000155,YUM,2016-12-30,-0.199500,2016-09-12,10,0.9,750,1,-0.199346
164221,0.531970,YUM,2018-03-05,-0.999677,2015-08-25,10,0.9,750,1,-1.531647
164222,0.277920,YUM,2017-11-06,-0.978144,2015-07-06,10,0.9,750,1,-1.256064
164223,-0.042968,YUM,2017-01-18,0.000000,2017-01-18,10,0.9,750,1,0.042968
164224,0.272480,YUM,2017-11-09,-0.940048,2015-08-06,10,0.9,750,1,-1.212528
164225,-0.084400,YUM,2018-02-08,-0.353490,2017-11-02,10,0.9,750,1,-0.269090
164226,-0.035804,YUM,2017-04-19,-0.344614,2016-12-09,10,0.9,750,1,-0.308810
164227,0.043770,YUM,2018-03-02,-0.415290,2017-10-26,10,0.9,750,1,-0.459060
164228,-0.052336,YUM,2017-03-21,-0.433875,2016-07-27,10,0.9,750,1,-0.381539
164229,0.098961,YUM,2017-03-02,-0.562982,2016-07-07,10,0.9,750,1,-0.661943


The three times there was a 0% return using my strategy, the money was only invested for 9, 6, or 8 days. All the other times had huge returns. It looks like the money just needs to be held for a certain period of time.

### What is the minimum amount of time the strategy needs to be used for?

In [32]:
best_strategy['Start'] = pd.to_datetime(best_strategy['Start'], format='%Y-%m-%d')
best_strategy['Ends'] = pd.to_datetime(best_strategy['End'], format='%Y-%m-%d')

c:\users\john maxi\anaconda3\envs\tensorflowenv\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
c:\users\john maxi\anaconda3\envs\tensorflowenv\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [33]:
(best_strategy[best_strategy['Maxi']<=0]['Ends'] - best_strategy[best_strategy['Maxi']<=0]['Start']).describe()

count                         469
mean     278 days 21:51:02.686567
std      254 days 17:08:45.998541
min               0 days 00:00:00
25%              61 days 00:00:00
50%             204 days 00:00:00
75%             449 days 00:00:00
max            1026 days 00:00:00
dtype: object

In [36]:
(best_strategy[best_strategy['Maxi']>0]['Ends'] - best_strategy[best_strategy['Maxi']>0]['Start']).describe()

count                          11
mean     135 days 19:38:10.909090
std      133 days 23:52:16.924253
min              21 days 00:00:00
25%              33 days 12:00:00
50%             119 days 00:00:00
75%             170 days 00:00:00
max             470 days 00:00:00
dtype: object

OK! Here is an interesting discovery, the simulations that were not profitable were short-term simulations, usually lasting less than 20 days. The profitable simulations usually lasted at least a year. We can deduce that on average, applying this stategy for a little less than a year would turn $1000 into into \$32512!!!

Next step is obviously to bet $1000 on this strategy using my ETrade account.

In [37]:
stock_data = quandlkey.quandl_stocks('SLGN', start_date=(2015,8,25), end_date=(2016,7,26))
sim = quandlkey.BollingerBands(stock_data['WIKI/SLGN - Adj. Close'], 
                                window=5, sell_per=.9, buy_amt=750, width=1,
                                balance=1000)
sim.trade()
base = sim.buy_and_hold()

In [38]:
%matplotlib
plt.style.use('seaborn-whitegrid')
fig, ax = plt.subplots(figsize=(12,6))
ax2 = ax.twinx()
ax.plot(stock_data.index, sim.results['Balance'], label='Cash', c='g')
ax.plot(stock_data.index, sim.results['Shares'], label='Shares', c='k')
ax2.plot(stock_data.index, base, label='Buy and Hold', c='b')
ax.plot(stock_data.index, sim.results['Shares']*sim.stock+sim.results['Balance'], label='My Strategy', c='r')#
ax.legend()
ax.set_ylabel('Account Value ($)', fontsize=20)
ax.set_xlabel('Data', fontsize=20)
ax.tick_params(axis = 'both', which = 'major', labelsize = 15)
ax.set_title('Maxi Band Trading', fontsize=30);

Using matplotlib backend: Qt5Agg


In [77]:
print(sim.stock)

Date
2015-08-25    24.980870
2015-08-26    24.990591
2015-08-27    25.578662
2015-08-28    25.602962
2015-08-31    25.524964
2015-09-01    24.740105
2015-09-02    24.905852
2015-09-03    25.261719
2015-09-04    24.944851
2015-09-08    25.612712
2015-09-09    25.685835
2015-09-10    25.422591
2015-09-11    25.378717
2015-09-14    25.286094
2015-09-15    25.788208
2015-09-16    26.129451
2015-09-17    26.061202
2015-09-18    25.383592
2015-09-21    25.783333
2015-09-22    25.266594
2015-09-23    24.910727
2015-09-24    24.886352
2015-09-25    24.930226
2015-09-28    24.983850
2015-09-29    25.013100
2015-09-30    25.368967
2015-10-01    25.481090
2015-10-02    25.832082
2015-10-05    26.392696
2015-10-06    26.129451
                ...    
2016-06-14    24.545664
2016-06-15    24.452147
2016-06-16    24.585039
2016-06-17    24.722854
2016-06-20    25.229812
2016-06-21    25.072311
2016-06-22    25.328251
2016-06-23    25.712161
2016-06-24    24.476757
2016-06-27    24.122378
2016-06-28 